# Create UC Functions

In [0]:
from databricks.connect import DatabricksSession
import mlflow
from dotenv import load_dotenv

spark = DatabricksSession.builder.getOrCreate()
load_dotenv('../.env')

DATABRICKS_HOST = os.getenv('host')
DATABRICKS_TOKEN = os.getenv('token')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(f"host: {DATABRICKS_HOST}")

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = 'fins_genai'
SCHEMA = 'agents'

In [0]:
# Create a new table with a markdown table that contains all the policy details
from pyspark.sql.functions import to_json, col, struct, concat_ws, lit

def create_md_col(df):
    # Select all columns and create a new column with Markdown table format
    header = "| " + " | ".join([c.replace("_", " ") for c in df.columns]) + " |"
    separator = "| " + " | ".join(["---"] * len(df.columns)) + " |"

    df_with_md = df.withColumn("policy_details", concat_ws(" | ", *[col(c).cast("string") for c in df.columns]))
    df_with_md = df_with_md.withColumn("policy_details", concat_ws("\n", lit(header), lit(separator),
                                                                   col("policy_details")))

    # Show the updated DataFrame
    df_with_md.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("fins_genai.agents.insurance_customer_profiles")

# Create UC Tools

## `search_policy_details_by_name()`

In [0]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.search_policy_details_by_name(
  search_name STRING
)
RETURNS TABLE (
  policy_details STRING
)
COMMENT 'Return a customer policy details with a given customer name'
LANGUAGE SQL
  RETURN
  SELECT
    policy_details
  FROM {CATALOG}.{SCHEMA}.insurance_customer_profiles
  where (CONCAT(first_name, ' ', last_name) = search_name)
  or (CONCAT(last_name, ' ', first_name) = search_name)
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
spark.sql("select * from fins_genai.agents.search_policy_details_by_name('Gloria Cook')").show(truncate=False)

## `search_policy_details_by_policy_number()`

In [0]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.search_policy_details_by_policy_number(
  search_name STRING
)
RETURNS TABLE (
  policy_details STRING
)
COMMENT 'Return a customer policy details with a given customer name'
LANGUAGE SQL
  RETURN
  SELECT
    policy_details
  FROM {CATALOG}.{SCHEMA}.insurance_customer_profiles
  where policy_number = search_name
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
spark.sql("select * from fins_genai.agents.search_policy_details_by_policy_number('102147984')") .show()

## `search_claims_details_by_policy_number()`

In [0]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.search_claims_details_by_policy_number(
  search_name STRING
)
RETURNS TABLE (
    clain_report STRING
)
COMMENT 'Return a claim details of a customer given policy number'
LANGUAGE SQL
  RETURN
  SELECT claim_report
  FROM {CATALOG}.{SCHEMA}.claims_details
WHERE policy_number = search_name
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
spark.sql("select * from fins_genai.agents.search_claims_details_by_policy_number('102148040')").show()

## `search_policy_doc()`

In [0]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.search_policy_doc(
    query STRING
)
RETURNS TABLE (
    policy STRING
)
COMMENT 'Perform search on insurance policy document to retrieve most relevant text for a given input query'
LANGUAGE SQL
  RETURN
  SELECT
   chunked_text as policy
  FROM vector_search(
    index => 'fins_genai.agents.policy_docs_chunked_files_vs_index',
    query_text => query,
    num_results => 3
  )
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
spark.sql("select * from fins_genai.agents.search_policy_doc('accident claim')").show()